This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [13]:
# get api key from your .env file
import os
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

In [2]:
#conda install -c conda-forge python-dotenv

Solving environment: ...working... done

# All requested packages already installed.

Retrieving notices: ...working... done

Note: you may need to restart the kernel to use updated packages.


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests
import collections

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
#1-Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 
#(keep in mind that the date format is YYYY-MM-DD).
params1=dict(start_date='2022-09-19', end_date='2022-09-19', api_key=API_KEY)
data=requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json', params=params1)

In [4]:
data_json=data.json()
data_json.keys()

dict_keys(['dataset_data'])

In [5]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
data_json['dataset_data']

{'limit': None,
 'transform': None,
 'column_index': None,
 'column_names': ['Date',
  'Open',
  'High',
  'Low',
  'Close',
  'Change',
  'Traded Volume',
  'Turnover',
  'Last Price of the Day',
  'Daily Traded Units',
  'Daily Turnover'],
 'start_date': '2022-09-19',
 'end_date': '2020-12-01',
 'frequency': 'daily',
 'data': [],
 'collapse': None,
 'order': None}

In [6]:
#2-Convert the returned JSON object into a Python dictionary.
params=dict(start_date='2017-01-01', end_date='2017-12-31', api_key=API_KEY)
res=requests.get('https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json', params=params)
json_dict=dict(res.json()['dataset_data'])

In [7]:
#3- Calculate what the highest and lowest opening prices were for the stock in this period.
open_data=dict()
change_data=dict()
closing_data=[]
volume_list=[]
for (date, open_price, high_price, low_price, close_price, change_price, traded_volume, turnover,
  last_price, daily_traded_units, daily_turnover) in json_dict['data']:
    if open_price!=None:
        open_data[date]=open_price
    if (high_price!=None) & (low_price!=None):
        change_data[date]=high_price-low_price
    closing_data.append((date, close_price))
    volume_list.append(traded_volume)
#find the highest and lowest opening price:   
max_open=max(open_data, key=open_data.get)
min_open=min(open_data, key=open_data.get)
print('The highest opening price was', open_data[max_open], 'on', max_open)
print('The lowest opening price was', open_data[min_open], 'on', min_open)

The highest opening price was 53.11 on 2017-12-14
The lowest opening price was 34.0 on 2017-01-24


In [8]:
#4-What was the largest change in any one day (based on High and Low price)?
max_change=max(change_data, key=change_data.get)
print('The largest change in one daya was', change_data[max_change], 'on', max_change)

The largest change in one daya was 2.8100000000000023 on 2017-05-11


In [9]:
#5-What was the largest change between any two days (based on Closing Price)?
closing_data.sort()
closing_data_change=[abs(closing_data[i+1][1] - closing_data[i][1]) for i in range(len(closing_data)-1)]
max(closing_data_change)

2.559999999999995

In [10]:
#6-What was the average daily trading volume during this year?
avg_vol=sum(volume_list) /len(volume_list)
print(avg_vol)

89124.33725490196


In [11]:
#7-(Optional) What was the median trading volume during this year.
#(Note: you may need to implement your own function for calculating the median.)
def med_vol(vol_list):
    vol_list.sort()
    if len(vol_list) % 2==1:
        middle=int((len(vol_list)-1)/2)
        return(vol_list[middle])
    else:
        mid_upper=int(len(vol_list)/2)
        mid_lower=mid_upper-1
        med=(vol_list[mid_upper]+vol_list[mid_lower])/2
        return(med)
med_vol(volume_list)

76286.0